# QC Tutorial

---

This interactive tutorial will show you a few ways to quality control data using LROSE using RadxQc and RadxPid/RadxRate. RadxPid and RadxRate allow users to censor non-weather echoes using the National Center for Atmospheric Research (NCAR) particle identification (PID) algorithm. 

RadxPid is one of three 'Echo' applications in LROSE. A visual comparison of RadxPid with RadxKdp and RadxRate and their parameter files is shown below. Each application has its own main parameter file that defines variable names and specifies the paths to the parameter file for each relevant sub-process. For example, the main RadxPid parameter file links to the specific differential phase (KDP) and PID parameter files (the PID parameter file links to the PID thresholds file).

<div>
<img src="../images/radx_echo_description.png" width="600"/>
</div>

---

*Note: this tutorial shows two examples of how to quality control radar data. Other workflows exist!*

---

## Tutorial Overview
### 1. Setup

#### Directory organization

The structure of the QC tutorial on JupyterHub is shown in the diagram below. The parent or base directory is "lrose-hub" and contains all of the notebooks, parameter files, and data for the workshop.

<div>
<img src="../images/qc_structure.png" width="500"/>
</div>

#### Raw data and parameter files

CfRadial data file that will be provided:
* cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc

Parameter files (will be filled out in this tutorial):

* RadxPid main params
* RadxRate main params
* RadxPid Kdp_specific params
* RadxPid Pid_specific params
* Pid Thresholds params (S-band, simultaneous transmit)
* RadxQc params

### 2. Run RadxPid/RadxRate - censor non-weather gates

* Run PID algorithm to identify gates which do not have weather
    * RadxPid/RadxRate

### 3. Run RadxQc - censor based on thresholds

* Censor gates where RHOHV and NCP don't meet thresholds
    * RadxQc

### 4. Plot PID and Rate

* Visualize results of RadxPid/RadxRate and RadxQc analysis using Py-ART

---

# 1. Setup
## Environment and packages

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. At the end we will plot the output using Py-ART.

In [ ]:
import os
import pyart
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

Next, we set up the directory structure to simplify our commands. If you are running this notebook at the LROSE workshop on JupyterHub, these paths go to the parent directory containing all the workshop resources and the LROSE binaries. 

**If you have downloaded this notebook, please modify BASE_DIR to work on your personal machine.**

* BASE_DIR: the base directory containing the directories for the notebooks, data, parameter files
* RADAR_NAME: the name of the radar used in this tutorial

In [ ]:
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
os.environ['RADAR_NAME'] = 'CHILL_S'
base_dir = os.environ['BASE_DIR']
raw_dir = os.environ['RAW_DIR']
radar_name = os.environ['RADAR_NAME']
!echo "Base directory: "$BASE_DIR
!echo "Radar name: "$RADAR_NAME

## Set up directories

We need to set up the required data directories. The radar data is provided on the JupyterHub. We delete any existing files and directories specific to this tutorial to ensure we're starting with clean directories and files.

In [ ]:
## make subdirectory within data for the raw data
!rm -rf ${BASE_DIR}/data/qc_guided/raw
!mkdir -p ${BASE_DIR}/data/qc_guided/raw

## make a directory for the parameter files
!rm -rf ${BASE_DIR}/params/qc_guided
!mkdir -p ${BASE_DIR}/params/qc_guided

We can examine the raw radar file using RadxPrint. RadxPrint can read CfRadial and other raw radar formats supported by LROSE that can later be converted by RadxConvert. It will also provide information on radar variables and sweep information. For example, we can look at the variables using the following command, piping the output into the head command.

In [ ]:
# print out the first 50 lines of RadxPrint output
!RadxPrint -f ${RAW_DIR}/qc_guided/*.nc | head -50


## Visually Inspect Data

First, let's look at a basic plot of the raw data using Py-ART. We notice that the data are quite noisy.

In [ ]:
# Read CfRadial file into radar object
inDir_raw = raw_dir+"/qc_guided/"
file_raw = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
chill_raw = pyart.io.read_cfradial(inDir_raw+file_raw)

# Plot raw data

displayRaw = pyart.graph.RadarDisplay(chill_raw)
figRaw = plt.figure(1, (12, 5))

# DBZ (input)

axDbz = figRaw.add_subplot(121)
displayRaw.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRaw.plot_range_rings([50, 100, 150])
displayRaw.plot_cross_hair(150.)
displayRaw.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axZdr = figRaw.add_subplot(122)
displayRaw.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRaw.plot_range_rings([50, 100, 150])
displayRaw.plot_cross_hair(150.)
displayRaw.set_limits(xlim=(-150,150),ylim=(-150,150))

# 2.  Run RadxPid - censor non-weather gates

## PID Algorithm

The NCAR PID algorithm uses polarimetric radar data to classify radar gates into 17 distinct categories, 3 of which are considered non-weather categories: insects, second-trip echoes, and clutter. RadxPid and RadxRate have the capability to censor gates that are not weather in the written output files. Thus, we will run the PID algorithm to censor the data. In order to run the PID, we first must calculate KDP from the PHIDP field in the raw files.

Since censoring does not care too much about the sounding quality (used for melting level identification), this tutorial will use a rough sounding from Denver 3 hours before this radar file, which will be included in the PID thresholds file. 

<div class="alert alert-block alert-info"> <b>Task: Check command line options of RadxPid.</b> 
    <br>
    LROSE applications often let you link to the input files and set the output directory on the command line. Let's check that those options exist (e.g., -f and -outdir). Type the following command into the open cell below and run it!
    <br>
    <code lang="bash">!RadxPid -h</code>
</div>

#### 2.1 Generate the parameter files
First, we need to generate the parameter file! LROSE applications use the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. 

As shown in the image at the beginning of the notebook, RadxPid requires 3 parameter files to run: 
* a main parameter file that sets variable names and file paths
* a KDP-specific parameter file that determines how KDP is calculated
* a PID-specific parameter file that specifies the location of the fuzzy logic thresholds. 

<div class="alert alert-block alert-info"> <b>Task: Create the RadxPid parameter files.</b> 
<br>
Type the following commands into the open cell below and run it!
<br>
<code lang="bash">!RadxPid -print_params > ${BASE_DIR}/params/qc_guided/RadxPid_main_params</code>
    
<br>
<code lang="bash">!RadxPid -print_params_kdp > ${BASE_DIR}/params/qc_guided/RadxPid_Kdp_params</code>

<code lang="bash">!RadxPid -print_params_pid > ${BASE_DIR}/params/qc_guided/RadxPid_Pid_params</code>
</div>

#### 2.2 Fill out the RadxPid parameter files

Now we will step through and edit the parameter files.

**RadxPid Main Parameter file (RadxPid_main_params)**

First, we'll start with the parameters associated with the necessary file paths. Since we determined earlier that we can link to the input files and output directory, we don't need to modify them here, but we've listed the line numbers for reference.

| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 98 |Path to input data, if not already specified on the command line.|
|KDP_params_file_path| 270 |Path to Kdp-specific parameter file.|
|PID_params_file_path| 287 |Path to PID-specific parameter file.|
|output_dir| 757 |Path where output files are written, if not already specified on command line with -outdir.|

<div class="alert alert-block alert-info"> <b>Task: change the kdp_params_file_path and pid_params_file_path parameters.</b> 
    <br>
    We generated the parameter files in the previous task - add those paths to the parameter file so RadxPid knows where to look.

1. Set <code lang="bash">KDP_params_file_path = "/home/jovyan/lrose-hub/params/qc_guided/RadxPid_Kdp_params"</code>
    
2. Set <code lang="bash">PID_params_file_path = "/home/jovyan/lrose-hub/params/qc_guided/RadxPid_Pid_params"</code>

</div>

---

Next, we'll look at the raw file to find which variables are available and what the variable names are.

| Parameter | Line # | Description |
| --- | --- | --- |
|SNR_available| 160 |Specify if SNR is available.|
|SNR_field_name| 170 |SNR variable name.|
|DBZ_field_name| 192 |DBZ variable name.|
|ZDR_field_name| 200 |ZDR variable name.|
|PHIDP_field_name| 208 |PHIDP variable name.|
|RHOHV_field_name| 216 |RHOHV variable name.|
|LDR_available| 224 |Specify if LDR is available.|
|LDR_field_name| 232 |LDR variable name.|

<div class="alert alert-block alert-info"> <b>Task: determine if SNR and LDR are present and grab the variable names for SNR, DBZ, ZDR, PHIDP, RHOHV, and LDR.</b> 
    
We can use ncdump or RadxPrint to examine the variable names in our radar files. To reduce the application output, you can also pipe information to commands like <code lang="bash">grep</code> or <code lang="bash">head</code>. 
 
<i>Use **one** of the ncdump or RadxPrint commands below in the next Jupyter cell. If you prefer to run them in a terminal window, remove the "!" symbol, which is just for Jupyter notebooks.</i>

<code lang="bash">!ncdump -h ${RAW_DIR}/qc_guided/cfrad*.nc </code>

<code lang="bash">!RadxPrint -f ${RAW_DIR}/qc_guided/cfrad*.nc</code>
 

Hint: The CHILL radar has an LDR variable for each polarization, but doesn't typically behave in a way RadxPid expects, so set LDR_available to FALSE.

</div>

---

RadxPid allows for use of attenuation-corrected fields in the PID. This is less of an issue with CHILL, which is an S-band radar, but it's good practice to account for attenutation in the PID.

| Parameter | Line # | Description |
| --- | --- | --- |
|PID_use_attenuation_corrected_fields| 311 |Specify whether to use attenuation-corrected DBZ and ZDR in PID.|

<div class="alert alert-block alert-info"> <b>Task: use attenuation-corrected fields.</b> 
    
Set <code lang="bash">PID_use_attenuation_corrected_fields = TRUE;</code>

</div>

---

Finally, we define the output variables and censor non-weather gates. Variables written to output files are divided into two categories: output fields (those variables created in RadxPid) and copy-through fields (variables from the input files that are copied to the output files). These variables are set in two different locations in the parameter file. 

<i>The line numbers listed here correspond to the default parameter file, but will change depending on whether additional variables are added.</i>

| Parameter | Line # | Description |
| --- | --- | --- |
|output_fields| 367 |Set which output fields are written to the output files (<code lang="bash">do_write</code>) and whether non-weather PID gates are censored (<code lang="bash">censor_non_weather</code>). The list of available output options are listed above the parameter.|
|copy_selected_input_fields_to_output| 478 |Tell RadxPid whether to copy through any fields to the output files.|
|copy_fields| 498 |Set which fields from the input files are written in the output files. <code lang="bash">input_name</code> must match the field name in the input file, <code lang="bash">output_name</code> is determined by the user (but is often the same as the original, and <code lang="bash">censor_non_weather</code> determines whether non-weather PID gates are censored for that variable.|

Of the fields calculated in RadxPid, we'll write out KDP, the PID, and the attenuation-corrected fields (DBZ_ATTEN_CORRECTED, ZDR_ATTEN_CORRECTED). We'll copy a few of the input fields to the output files as well. We will also censor non-weather gates.

<div class="alert alert-block alert-info"> <b>Task: specify output variables and censor non-weather gates for all variables except PID.</b> 
    
1. Output fields: write PID, KDP, DBZ_ATTEN_CORRECTED, and ZDR_ATTEN_CORRECTED. Censor non-weather for all fields except PID.

2. Set <code lang="bash">copy_selected_input_fields_to_output = TRUE;</code>
   
3. Copy-through fields: copy through DBZ, ZDR, and VEL from the input files (at least). Censor non-weather for all fields.

</div>


    
**RadxPid KDP-Specific Parameter file (RadxPid_Kdp_params)**

Usually, the default settings for KDP calculation are fine, but two commonly modified parameters are the filter length and the method for handling phase shift on backscatter.

| Parameter | Line # | Description |
| --- | --- | --- |
|KDP_fir_filter_len| 63 |Filter length used for KDP calculation.|
|KDP_psob_method| 108 |Specify method to remove phase shift on backscatter.|
          
We will use the defaults in this example, so no need to modify this parameter file.

**RadxPid PID-Specific Parameter file (RadxPid_Pid_params)**

The most important components of the PID algorithm are the PID thresholds file and a relevant sounding for estimating the melting level (i.e., distinguishing between liquid and frozen hydrometeors).

The PID thresholds files, particularly for S-band radars, have been tested in many environments, so those are provided for users on the LROSE wiki.

Soundings are either provided through 1) database files for each radar location and at a user-specified temporal frequency, or 2) a single sounding that is added to the PID thresholds file.

| Parameter | Line # | Description |
| --- | --- | --- |
|PID_thresholds_file_path| 25 |Path to fuzzy logic PID thresholds file.|
|PID_use_soundings_from_spdb| 259 |Specify whether soundings are in Spdb format, otherwise sounding found in fuzzy logic file.|
|PID_sounding_spdb_url| 267 |Path to Spdb soundings.|
|PID_sounding_location_name| 289 |Name of sounding location.|

In this case, since we care most about the non-meteorological categories for censoring data, we will add a single sounding from Denver roughly 3 hours before our raw radar file. This is generally not recommended for midlatitude weather where the melting level could vary a lot or when wanting to get realistic PID categories, but can work for simple data censorship.

<div class="alert alert-block alert-info"> <b>Task: download and direct RadxPid to the PID thresholds file.</b> 

1. Download the S-band PID thresholds file.

<code lang="bash">!wget http://wiki.lrose.net/images/d/de/Pid_thresholds.sband.shv.txt</code>

2. Move the file to a better directory.

<code lang="bash">!mv ./Pid_thresholds.sband.shv.txt ${BASE_DIR}/params/qc_guided/</code>

3. Add the new path of the thresholds file to the parameter file.

4. Set <code lang="bash">PID_use_soundings_from_spdb = FALSE;</code>

</div>

**PID thresholds file**

Finally, we'll add the sounding into the thresholds file.

| Parameter | Line # | Description |
| --- | --- | --- |
|Tpf| 57 |If not using Spdb soundings, a single sounding is drawn from this variable (altitude (km), temperature (C)).|

Here, we'll use the following simple sounding.

| Altitude (km) | Temperature |
| --- | --- |
| 0 | 25 |
| 1.8 | 30.4 |
| 4.0 | 8.4 |
| 4.9 | 2.4 |
| 5.8 | -4.9 |
| 7.0 | -14.9 |
| 10.0 | -38.6 |

<div class="alert alert-block alert-info"> <b>Task: add the sounding to the PID thresholds file.</b> 

1. Comment out the existing sounding by adding a "#" at the beginning of the line.

2. Add the sounding in the following format in the line below: ( alt1, temp1 ) ( alt2, temp2 ) ( alt3, temp3 ) ( alt4, temp4 ) ... .

</div> 


****

**Now we're ready to run RadxPid!**

Here, <code lang="bash">-params</code> links to the main RadxPid parameter file, <code lang="bash">-f</code> links to the specific files we want to process, and <code lang="bash">-outdir</code> indicates where RadxPid should write the output files.

In [ ]:
# delete any existing data
!rm -rf ${BASE_DIR}/data/qc_guided/radxpid
!mkdir -p ${BASE_DIR}/data/qc_guided/radxpid

# run RadxPid 
!RadxPid -params ${BASE_DIR}/params/qc_guided/RadxPid_main_params -f ${RAW_DIR}/qc_guided/*.nc -outdir ${BASE_DIR}/data/qc_guided/radxpid
#!RadxRate -params ${BASE_DIR}/params/qc_guided/RadxRate_main_params -f ${RAW_DIR}/qc_guided/*.nc -outdir ${BASE_DIR}/data/qc_guided/radxrate


# 3. Run RadxQc - censor based on thresholds

## RadxQc algorithm
RadxQc includes a few quality control algorithms, including KDP calculation, the PID algorithm, removing RLAN interference, and censoring based on variable thresholds. 

In this example, we will censor based on the values of RHOHV and NCP in this tutorial. We will only retain gates where RHOHV values range between 0.8 - 1.1 and NCP values range between 0.4 - 1.1. In addition, to remove short runs of noise, we will remove gates where the run of uncensored gates totals less than 5 gates. We will write DBZ, ZDR, PHIDP, RHOHV, and NCP to the output file, where only DBZ, PHIDP, and ZDR have been censored.

<div class="alert alert-block alert-info"> <b>Task: Check command line options of RadxQc</b> 
    <br>
    LROSE applications often let you link to the input files and set the output directory on the command line. Let's check that those options exist (e.g., -f and -outdir). Type the following command into the open cell below and run it!
    <br>
    <code lang="bash">!RadxQc -h</code>
</div>



#### 3.1 Generate the parameter files
First, we need to generate the parameter file! LROSE applications use the <code lang="bash">-print_params</code> flag to generate parameter files with the defaults. We just need to specify the location and name of the parameter file. RadxQc only requires one parameter file.

<div class="alert alert-block alert-info"> <b>Task: Create the RadxQc parameter files.</b> 
<br>
Type the following commands into the open cell below and run it!
<br>
<code lang="bash">!RadxQc -print_params > ${BASE_DIR}/params/qc_guided/RadxQc_params</code>
</div>


#### 3.2 Fill out the RadxQc parameter file (RadxQc_params)

Similar to RadxPid, one of the first tasks is to tell the application what the variable names are. Similar to RadxPid, we'll link to the input file directly on the command line, so we <i>do not</i> need to set the input directory.

In this example, we will just censor data based on values of RHOHV and NCP, so we don't need to worry about all of the variables except those that we'll 

| Parameter | Line # | Description |
| --- | --- | --- |
|input_dir| 118 |Path to input data, if not specified on the command line.|
|DBZ_field_name| 378 |DBZ variable name.|
|VEL_field_name| 388 |VEL variable name.|
|WIDTH_available| 399 |Specify if WIDTH is available.|
|WIDTH_field_name| 407 |WIDTH variable name.|
|NCP_available| 418 |Specify if NCP is available, in this example FALSE.|
|NCP_field_name| 426 |NCP variable name.|
|SNR_available| 437 |Specify if SNR is available, in this example FALSE.|
|SNR_field_name| 447 |SNR variable name.|
|ZDR_available| 469 |Specify if ZDR is available.|
|ZDR_field_name| 477 |ZDR variable name.|
|LDR_available| 485 |Specify if LDR is available, in this example FALSE.|
|LDR_field_name| 495 |LDR variable name.|
|PHIDP_available| 503 |Specify if PHIDP is available.|
|PHIDP_field_name| 511 |PHIDP variable name.|
|RHOHV_available| 519 |Specify if RHOHV is available.|
|RHOHV_field_name| 527 |RHOHV variable name.|

<div class="alert alert-block alert-info"> <b>Task: determine if RHOHV and NCP are present and set the variable names.</b> 
    
We can use ncdump or RadxPrint to examine the variable names in our radar files. To reduce the application output, you can also pipe information to commands like <code lang="bash">grep</code> or <code lang="bash">head</code>. 
 
<i>Use **one** of the ncdump or RadxPrint commands below in the next Jupyter cell. If you prefer to run them in a terminal window, remove the "!" symbol, which is just for Jupyter notebooks.</i>

<code lang="bash">!ncdump -h ${RAW_DIR}/qc_guided/cfrad*.nc </code>

<code lang="bash">!RadxPrint -f ${RAW_DIR}/qc_guided/cfrad*.nc</code>
 

</div>

---

Finally, now we'll specify the method of quality control, set our thresholding conditions, and tell RadxQc which fields we'd like to write to the output files and apply the censoring.

| Parameter | Line # | Description |
| --- | --- | --- |
|compute_pid| 1334 |Tell RadxQc whether to run the PID algorithm for censoring, in this example FALSE.|
|censoring_input_fields| 1877 |Specify variables, variable thresholds, and combination method used to censor data.|
|input_field_censoring_min_valid_run| 1906 |Minimum valid run of non-censored gates.|
|censored_output_fields| 1946 |Specify the fields to be copied from the input files, censored, and written to the output files.|

<div class="alert alert-block alert-info"> <b>Task: Set censoring method, specify thresholds, and specify output fields.</b> 
    
1. compute_pid: tell RadxQc to not run the PID algorithm (FALSE)
   
2. censoring_input_fields: censor data based on RHOHV and NCP, retaining data where 0.8 <= RHOHV <= 1.1 and 0.4 <= NCP <= 1.1

3. input_field_censoring_min_valid_run: set to 5

4. censored_output_fields: censor and write DBZ, ZDR, PHIDP, RHOHV, and NCP to output files. Set <code lang="bash">apply_input_field_censoring = TRUE;</code>.


Hint: Add commas between entries to censored_output_fields, like below.

<code lang="bash">censored_output_fields = {
  {
    input_name = "DBZ",
    output_name = "DBZ",
    apply_rlan_censoring = FALSE,
    apply_seaclut_censoring = FALSE,
    apply_pid_censoring = FALSE,
    apply_input_field_censoring = TRUE
  }
  ,
  {
    input_name = "ZDR",
    output_name = "ZDR",
    apply_rlan_censoring = FALSE,
    apply_seaclut_censoring = FALSE,
    apply_pid_censoring = FALSE,
    apply_input_field_censoring = TRUE
  }
}
</code>

</div>

****

**Now we're ready to run RadxQc!**

Here, *-params* provides the link to the main RadxQc parameter file, *-f* provides the link to the files we want to process, and *-outdir* indicates where RadxQc should write the final files.

In [ ]:
!rm -rf ${BASE_DIR}/data/qc_guided/radxqc
!mkdir -p ${BASE_DIR}/data/qc_guided/radxqc

# run RadxQc
!RadxQc -params ${BASE_DIR}/params/qc_guided/RadxQc_params -f ${RAW_DIR}/qc_guided/*.nc -outdir ${BASE_DIR}/data/qc_guided/radxqc


# 4. Plot censored CHILL output

## 4.1 Censoring using the PID (RadxPid)

To visualize the output in the notebook, we will use Py-ART. 

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc_guided/radxpid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
rate_kmhx = pyart.io.read_cfradial(inDir+file)
rate_kmhx.info('compact')


We can create a colormap for visualizing the PID.

In [ ]:
pidmap = np.array([[0.12156862745098039, 0.46666666666666667, 0.70588235294117652, 1.0],
              [0.68235294117647061, 0.7803921568627451, 0.90980392156862744, 1.0],
              [0.59607843137254901, 0.87450980392156863, 0.54117647058823526, 1.0],
              [0.45490196078431372, 0.7686274509803922, 0.46274509803921571, 1.0],
              [0.17254901960784313, 0.62745098039215685, 0.17254901960784313, 1.0],
              [0.83921568627450982, 0.15294117647058825, 0.15686274509803921, 1.0],
              [1.0, 0.59607843137254901, 0.58823529411764708, 1.0],
              [1.0, 0.49803921568627452, 0.054901960784313725, 1.0],
              [1.0, 0.73333333333333328, 0.47058823529411764, 1.0],
              [0.61960784313725492, 0.85490196078431369, 0.89803921568627454, 1.0],
              [0.090196078431372548, 0.74509803921568629, 0.81176470588235294, 1.0],
              [0.61176470588235299, 0.61960784313725492, 0.87058823529411766, 1.0],
              [0.32156862745098042, 0.32941176470588235, 0.63921568627450975, 1.0],
              [0.859375, 0.859375, 0.859375, 1.0],
              [0.66015625, 0.66015625, 0.66015625, 1.0],
              [0.41015625, 0.41015625, 0.41015625, 1.0],
              [0.0, 0.0, 0.0, 1.0],],'f')
my_cmap2 = colors.ListedColormap(pidmap, name='ncar_pid')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


Now we'll make the same plot, but with the attenuation-corrected fields.

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kmhx)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="Attenuation-Corrected DBZ",cmap="pyart_HomeyerRainbow")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR_ATTEN_CORRECTED', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="Attenuation-Corrected ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


## 4.2 Censoring using the variable thresholds (RadxQc)

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc_guided/radxqc/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
qc_chill = pyart.io.read_cfradial(inDir+file)
qc_chill.info('compact')

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(qc_chill)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('ZDR', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR (dB)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# RHOHV

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RHOHV', 0, vmin=0.8, vmax=1.05,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RHOHV")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCP

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('NCP', 0, vmin=0.4, vmax = 1.1,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="NCP",
    mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

figRate.tight_layout()

plt.show()


# 5. Bonus fun - run let's run RadxQc and then RadxPid!

Users may already have existing quality control procedures that they like to use. User preferences with respect to noise, speckling, etc. will vary and often multiple processing steps are involved. Pre-processing using Py-ART or CSU RadarTools can also be fed into RadxPid.

Let's feed the output from RadxQc into RadxPid.

Since our parameter files use the same variable names and we've used the command line flags to specify the input and output paths, we can make simple modifications to our application call.

Changes made below:
* Create a new output directory <code lang="bash">data/qc_guided/radxqc_then_pid</code> 
* Change the <code lang="bash">-f</code> flag to point to the RadxQc output <code lang="bash">data/qc_guided/radxqc/</code>
* Change the <code lang="bash">-outdir</code> flag to point to the new output directory

In [ ]:
!rm -rf ${BASE_DIR}/data/qc_guided/radxqc_then_pid
!mkdir -p ${BASE_DIR}/data/qc_guided/radxqc_then_pid

# run RadxPid using RadxQc output
!RadxPid -params ${BASE_DIR}/params/qc_guided/RadxPid_main_params -f ${BASE_DIR}/data/qc_guided/radxqc/20210815/*.nc -outdir ${BASE_DIR}/data/qc_guided/radxqc_then_pid
#!RadxRate -params ${BASE_DIR}/params/qc_guided/RadxRate_main_params -f ${BASE_DIR}/data/qc_guided/radxqc/20210815/*.nc -outdir ${BASE_DIR}/data/qc_guided/radxqc_then_pid

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc_guided/radxqc_then_pid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
pid_then_qc = pyart.io.read_cfradial(inDir+file)
pid_then_qc.info('compact')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(pid_then_qc)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ_ATTEN_CORRECTED', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="Attenuation-Corrected DBZ",cmap="pyart_HomeyerRainbow")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR_ATTEN_CORRECTED', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="Attenuation-Corrected ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


# 6. More bonus fun - combine RadxRate with external QC algorithms

Here in this example, we provide some Python QC code that uses Py-ART and CSU RadarTools to do an initial QC of the data. Then we feed the output into RadxRate, similar to what we did in the last section.

In [ ]:
def run_qc_algs(radar, sdthresh = 13):
    
    import numpy as np
    from csu_radartools import (csu_kdp, csu_misc)
    import copy
    from astropy.convolution import (Gaussian2DKernel, convolve)

    # ingest base variables
    rho_field = radar.fields['RHOHV']['data']
    phi_field = copy.deepcopy(radar.fields['PHIDP']['data'])
    dbz_field = radar.fields['DBZ']['data']
    range_2d = np.tile(radar.range['data'],(radar.time['data'].shape[0],1))
    fv = radar.fields['PHIDP']['_FillValue']
        
    # ingest remaining variables
    ncp_field = radar.fields['NCP']['data']
    ldrh_field = radar.fields['LDRH']['data']
    ldrv_field = radar.fields['LDRV']['data']
    
    sd_lin = csu_kdp.calc_kdp_bringi(phi_field, dbz_field, range_2d, bad=fv, gs=150., window=7)[2]
        
    # smooth ncp for thresholding
    sm_g_rad = 5
    gauss_kernel = Gaussian2DKernel(sm_g_rad, sm_g_rad)
    ncp_smooth = convolve(ncp_field, gauss_kernel)

    # combine thresholds for feeding data into despeckle
    ldr_bad = (ldrh_field > -5.) | (ldrv_field > -5.) | ((dbz_field < 0.) & (ldrh_field > -15.))
    ncp_bad = (ncp_smooth < 0.4) | (ncp_smooth > 1.1)
    rho_bad = (rho_field < 0.7) | (rho_field > 1.1)
    qc_comb = ((sd_lin > sdthresh) & (dbz_field < 10.)) | ldr_bad | ncp_bad | rho_bad 

    # despeckle
    desp = csu_misc.despeckle(np.where(qc_comb, np.nan, dbz_field), bad=fv, ngates=4)
    
    # combine masks
    bad = (dbz_field == fv) | qc_comb | desp
    
    return(sd_lin, desp, bad)

In [ ]:
def clean_data_add_vars(radar,bad):
    
    import numpy as np
    
    vars = list(radar.fields.keys())
    
    for x in vars:
        
        var = radar.fields[x]['data']
        
        # CHILL's fill value isn't getting processed correctly... manually correcting it
        if np.isnan(radar.fields[x]['_FillValue']):
            fv = -9e33
        else:
            fv = radar.fields[x]['_FillValue']

        var_copy = np.ma.masked_values(np.where(bad,fv,var),fv)
        radar.add_field_like(x, x, var_copy, replace_existing=True)

        if fv != radar.fields[x]['_FillValue']:
            radar.fields[x]['_FillValue'] = fv
            print(radar.fields[x]['_FillValue'])

    return(radar)

In [ ]:
def save_cfradial(radar,inDir,subdir,file):
    
    import os
    from pyart.io import write_cfradial

    path = inDir+subdir
    
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    
    if not isExist:
      
      # Create a new directory because it does not exist 
      os.makedirs(path)
      print("The new directory is created!")
      
    write_cfradial(path+file, radar)

In [ ]:
# run thresholding and despeckling
sd_lin, desp, bad1 = run_qc_algs(chill_raw, 18)

# apply masks to radar objects 
radar1 = clean_data_add_vars(chill_raw, bad1)

# save new cfradial file with new variables
save_cfradial(radar1, base_dir, '/data/qc_guided/python/', file_raw)

Now we can run that output through RadxRate and plot it using Py-ART.

In [ ]:
!rm -rf ${BASE_DIR}/data/qc_guided/python_then_pid
!mkdir -p ${BASE_DIR}/data/qc_guided/python_then_pid

# run RadxPid using RadxQc output
!RadxPid -params ${BASE_DIR}/params/qc_guided/RadxPid_main_params -f ${BASE_DIR}/data/qc_guided/python/*.nc -outdir ${BASE_DIR}/data/qc_guided/python_then_pid
#!RadxRate -params ${BASE_DIR}/params/qc_guided/RadxRate_main_params -f ${BASE_DIR}/data/qc_guided/python/*.nc -outdir ${BASE_DIR}/data/qc_guided/python_then_pid

In [ ]:
# Read CfRadial file into radar object
inDir = base_dir+"/data/qc_guided/python_then_pid/20210815/"
file = "cfrad.20210815_032510.928_to_20210815_032947.216_CSU-CHILL_SUR.nc"
pid_then_qc = pyart.io.read_cfradial(inDir+file)
pid_then_qc.info('compact')


In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(pid_then_qc)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ",cmap="pyart_HomeyerRainbow")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# ZDR

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('ZDR', 0, vmin=-1., vmax=5.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="ZDR",
    cmap="nipy_spectral")
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0, vmin=0.5, vmax = 17.5,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = my_cmap2, mask_outside=True)
displayRate.plot_range_rings([50, 100, 150])
displayRate.plot_cross_hair(150.)
displayRate.set_limits(xlim=(-150,150),ylim=(-150,150))

# plot all 17 PID categories
pid_cbar = displayRate.cbs[3]
#pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13])
#pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice'])
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()


Each user and organization will have different QC needs, and you'll notice that all of these different methods produced different results. There are many different workflows depending on the available fields.